# WINE QUALITY DATASET

Este notebook forma parte del proyecto **PC1: Análisis de Calidad de Vinos**, cuyo objetivo es analizar las características que determinan la calidad de los vinos tintos y blancos utilizando el [Wine Quality Dataset](http://archive.ics.uci.edu/dataset/186/wine+quality).

### **Objetivo de este notebook**
En este notebook se llevarán a cabo los siguientes pasos:
1. Descargar programáticamente los datasets de vinos tintos y blancos desde el repositorio oficial y asegurarse de que estén disponibles en el entorno de trabajo como archivos CSV.
2. Combinar los datasets en un único dataframe, añadiendo una columna que indique el tipo de vino (`red` o `white`), y verificar la cantidad de registros y las variables disponibles.
3. Realizar un análisis estadístico o inspección visual de cada columna numérica para identificar y manejar valores atípicos y ausentes.
4. Almacenar los datos limpios en una base de datos SQLite para garantizar persistencia y eficiencia.
5. Realizar consultas SQL sobre los datos almacenados, incluyendo:
   - El promedio de calidad (`quality`) por tipo de vino (`type`).
   - El conteo de vinos con nivel de alcohol superior a 10.5, agrupados por tipo.
   - El conteo de vinos por nivel de acidez (`fixed acidity`), agrupados en rangos específicos.
6. Exportar los resultados de una consulta seleccionada en formato JSONLines para su potencial uso en una base de datos noSQL como MongoDB.
7. Inspeccionar las características de los vinos tintos y blancos de mayor calidad (`quality`) utilizando técnicas estadísticas y gráficas.

## 1.1 Configuración del entorno

Este notebook utiliza las dependencias definidas en el archivo `requirements.txt`.
El entorno virtual asociado es `PC1`. Asegúrate de que esté activado antes de ejecutar este notebook.

In [1]:
#!python --version
#!pip list

In [2]:
# Mostrar el contenido del archivo requirements.txt
with open("../requirements.txt", "r") as f:
    print(f.read())

#
# This file is autogenerated by pip-compile with Python 3.13
# by the following command:
#
#    pip-compile requirements.in
#
contourpy==1.3.1
    # via matplotlib
cycler==0.12.1
    # via matplotlib
fonttools==4.55.3
    # via matplotlib
kiwisolver==1.4.8
    # via matplotlib
matplotlib==3.10.0
    # via
    #   -r requirements.in
    #   seaborn
numpy==2.2.1
    # via
    #   -r requirements.in
    #   contourpy
    #   matplotlib
    #   pandas
    #   seaborn
packaging==24.2
    # via matplotlib
pandas==2.2.3
    # via
    #   -r requirements.in
    #   seaborn
pillow==11.1.0
    # via matplotlib
pyparsing==3.2.1
    # via matplotlib
python-dateutil==2.9.0.post0
    # via
    #   matplotlib
    #   pandas
pytz==2024.2
    # via pandas
seaborn==0.13.2
    # via -r requirements.in
six==1.17.0
    # via python-dateutil
tzdata==2024.2
    # via pandas



## 1.2 Descarga de los datasets

**Opciones para acceder al dataset**

(`Enlace directo`): Se creó la carpeta data para descargar los datos manualmente desde las URLs del repositorio UCI. Sin embargo, si los datos son actualizados en el repositorio, no se sincronizan automáticamente.

(`Librería ucimlrepo`): Automatiza la descarga y carga de datos como DataFrames, incluyendo metadatos. Requiere instalar una librería adicional.

(`Repositorio GitHub`): Clona el repositorio completo de UCI para explorar múltiples datasets. Útil para trabajos más amplios, pero menos específico.

(`Usando WebScrapping`): A través de selenium se descargarán los datasets sin necesidad de entrar en el enlace 

Dado que los datasets descargados manualmente incluyen explícitamente la diferenciación entre vinos tintos y blancos en archivos separados (winequality-red.csv y winequality-white.csv), se utilizarán estos archivos para realizar la combinación de datos. Pero los descargaremos usando webScrapping

Aunque la librería ucimlrepo organiza los datos en un formato estándar, no proporciona información explícita para diferenciar entre vinos tintos y blancos. Este análisis será abordado como un ejercicio extra al final del proyecto, donde se intentará identificar las diferencias entre ambos tipos de vino basándonos en características fisicoquímicas

El dataset se encuentra en una carpeta zip en la URL proporcionada.

Se carga Selenium para acceder al dataset a traves de XPATH

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from time import sleep
import requests
from bs4 import BeautifulSoup

# Ruta al ChromeDriver
service = Service(executable_path="C:/Users/Oscar/Documents/MBIT_Oscar/Ruta_Selenium/chromedriver-win64/chromedriver.exe")

# Opciones para Chrome
options = webdriver.ChromeOptions()
options.add_argument('--incognito')

# Iniciar el navegador
driver = webdriver.Chrome(service=service, options=options)
driver.get("http://archive.ics.uci.edu/dataset/186/wine+quality")

ModuleNotFoundError: No module named 'selenium'

In [ ]:
dataset_download = driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/div[1]/main/div/div[2]/div[1]/a")
dataset_download.click()

# Esperar un poco para que la descarga termine
sleep(5)

driver.close()
# este paso descarga el archivo ZIP que contiene ambos datasets en la carpeta Downloads de Windows.

In [ ]:
import os
import zipfile
from zipfile import ZipFile 

#  Ruta de descarga en la carpeta Downloads
download_folder = "C:/Users/Oscar/Downloads"
zip_filename = "wine+quality.zip"
zip_filepath = os.path.join(download_folder, zip_filename)

# Directorio donde queremos guardar los archivos extraídos
destination_folder = "C:/Users/Oscar/Documents/MBIT_Oscar/MBIT_202501_Proyecto_Consolidacion_1/data"

# Verificar si el archivo ZIP existe antes de continuar
if os.path.exists(zip_filepath):
    print(f"Archivo ZIP encontrado: {zip_filepath}")

    # Extraer el contenido
    with ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(destination_folder)
        print(f"Archivos extraídos en {destination_folder}")

    # Mover el archivo ZIP a la carpeta de datos para almacenamiento (opcional)
    shutil.move(zip_filepath, os.path.join(destination_folder, zip_filename))
    print(f"Archivo ZIP movido a {destination_folder}")

else:
    print(" No se encontró el archivo ZIP en la carpeta Downloads. Verifica que se haya descargado correctamente.")
 

# 2. Combinar los datos

En este paso se combinarán los datasets de vinos tintos y blancos en un único DataFrame. Se añadirá una columna adicional que indique el tipo de vino (red o white) para diferenciarlos. Además, se analizará la cantidad total de registros y las variables disponibles, identificando sus tipos y características.

In [ ]:
#leemos los datasets, utilizamos el sep ; ya que en el dataset viene separado por ;
df_red_wine = pd.read_csv('../data/winequality-red.csv', sep = ';')
df_white_wine = pd.read_csv('../data/winequality-white.csv', sep = ';')

In [ ]:
#Primeros pediremos información de los datasets
df_red_wine.info()
df_white_wine.info()

Como podemos comprobar las columnas están separadas por ; por lo que procederemos a realizar una limpieza y preparación de datos

Esto nos indica que no hay valores nulos

In [ ]:
#Crearemos la coumna type-wine para clasificar los que son tintos y blancos para después combinarlos
df_red_wine['type-wine'] = 'red'
df_white_wine['type-wine'] = 'white'

In [ ]:
# checkeamos que se ha creado
df_white_wine.sample(5)

In [ ]:
#Ahora combinamos los dos dataframes
df_wine_quality = pd.concat([df_red_wine, df_white_wine])
df_wine_quality.sample(5)

In [ ]:
df_wine_quality.info()
df_wine_quality.shape
# Comprobamos las columnas, nº de datos y tipo de datos

In [ ]:
df_wine_quality.columns # comprobamos que en las columnas no existen espacios en blanco antes o después de las comillas

In [ ]:
df_wine_quality.describe() # realizamos in informe estadístico

# 3. Filtramos los atípicos y manejar los ausentes

Hasta este punto, hemos descargado, limpiado y combinado los datasets de vinos tintos y blancos en un único DataFrame. También hemos realizado una inspección inicial de las características de los datos, verificando su estructura, valores nulos y estadísticas descriptivas.
Para continuar con el análisis exploratorio, es fundamental detectar la presencia de valores atípicos (outliers) en las variables numéricas y evaluar la distribución de los datos. Esto nos ayudará a comprender mejor la variabilidad de los datos y tomar decisiones sobre el preprocesamiento antes de aplicar modelos predictivos.


## Detección de Outliers con Boxplot
El boxplot es una herramienta visual que nos permite detectar valores atípicos dentro de cada variable. Se basa en los cuartiles y los bigotes para identificar puntos que están considerablemente alejados de la distribución central de los datos. Los valores atípicos pueden ser el resultado de errores de medición, datos mal ingresados o simplemente fenómenos extremos dentro de la distribución.

¿Por qué es importante detectar outliers?

- Pueden afectar la precisión de los modelos predictivos.
- Influyen en el cálculo de la media y la varianza.
- En algunos casos, pueden proporcionar información relevante sobre variaciones extremas en la calidad del vino.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Definir tamaño de figura y DPI alto
plt.figure(figsize=(12, 6), dpi=150)

# Crear el boxplot con mejoras
sns.boxplot(data=df_wine_quality, orient='h', width=0.7)

# Personalizar estilo y ejes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Valores", fontsize=14)
plt.ylabel("Variables", fontsize=14)
plt.title("Distribución de las Variables en el Dataset", fontsize=16, fontweight="bold")

# Mostrar la gráfica
plt.show()


## Análisis de Distribución: Normalidad de los Datos
Después de detectar los **valores atípicos**, analizaremos si los datos siguen una **distribución de los datos** utilizando **histogramas** y **curvas KDE (Kernel Density Estimation)**. La normalidad es un supuesto fundamental en muchas técnicas estadísticas y de Machine Learning, ya que:

¿Por qué queremos verificar la distribución normal de los datos?

- Si los datos son **normales**, algunos modelos como `regresión lineal o ANOVA` pueden aplicarse directamente sin transformaciones adicionales.
- Si los datos **no son normales**, puede ser necesario aplicar técnicas como `escalado logarítmico, Box-Cox o estandarización` para mejorar el rendimiento de ciertos modelos.
- La normalidad nos permite interpretar mejor las métricas de dispersión como la desviación estándar y la media.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Lista de columnas numéricas (excluyendo 'type-wine' porque es categórica)
numeric_columns = df_wine_quality.select_dtypes(include=['float64', 'int64']).columns

# Graficar histogramas de todas las columnas numéricas
for col in numeric_columns:
    plt.figure(figsize=(6, 4))
    sns.histplot(df_wine_quality[col], bins=30, kde=True)
    plt.title(f"Distribución de {col}")
    plt.xlabel(col)
    plt.ylabel("Frecuencia")
    plt.show()

In [ ]:
import scipy.stats as stats

# Generar Q-Q plots para todas las columnas numéricas
for col in numeric_columns:
    plt.figure(figsize=(6, 4))
    stats.probplot(df_wine_quality[col], dist="norm", plot=plt)
    plt.title(f"Q-Q plot de {col}")
    plt.show()

## Conclusión del Análisis Exploratorio
A partir del boxplot, observamos la presencia de varios valores atípicos (outliers) en múltiples variables. En particular, se pueden identificar valores extremos en características como free sulfur dioxide y total sulfur dioxide, lo cual indica que algunos vinos tienen concentraciones excepcionalmente altas de estos compuestos en comparación con el resto de la muestra.

Por otro lado, al analizar la distribución de los datos mediante histogramas, encontramos que la mayoría de las variables no siguen una distribución normal, sino que presentan una asimetría positiva (cola a la derecha). Esto sugiere que los valores altos son menos frecuentes y que una parte significativa de los datos se concentra en valores más bajos. Sin embargo, una excepción notable es el pH y la densidad, que sigue una distribución aproximadamente normal.


## Tratamiento de Valores Atípicos: Métodos IQR y Six Sigma
Al observar el boxplot, hemos identificado la presencia de valores atípicos en varias variables del dataset. Para asegurar que nuestro análisis y modelo sean robustos, procederemos con el tratamiento de estos valores utilizando dos técnicas comúnmente empleadas en la ciencia de datos.

Dado que nuestros datos **no siguen una distribución normal**, el método **IQR (Rango Intercuartílico)** es la opción más adecuada para tratar los valores atípicos. Esto se debe a que:

`IQR es un método robusto`, basado en los cuartiles, lo que lo hace menos sensible a la presencia de valores extremos en la distribución.  
`Six Sigma se basa en la desviación estándar`, una métrica que puede verse significativamente afectada por distribuciones no normales y la presencia de valores atípicos, lo que lo hace menos confiable en este contexto.  

Por lo tanto, utilizaremos **IQR para filtrar los valores atípicos** de manera más efectiva y sin comprometer la integridad del conjunto de datos. 

In [ ]:
#Antes de empezar a hacer un tratamiento de datos realizaremos una copia sobre la que trabajar
df_wine_clean = df_wine_quality.copy()

### Método 1: Rango Intercuartil (IQR – Interquartile Range)
El IQR (Interquartile Range) es una técnica basada en la estadística descriptiva que nos permite detectar y eliminar valores extremos en una distribución. Se basa en los percentiles Q1 (25%) y Q3 (75%) de los datos.

### ¿Cómo funciona el IQR?
1. Calculamos Q1 (percentil 25%) y Q3 (percentil 75%).
2. Calculamos el rango intercuartil (IQR) con la fórmula:
IQR=Q3−Q1
3. Definimos los límites superior e inferior:
    - Límite inferior: Q1−1.5×IQR
    - Límite superior: Q3+1.5×IQR
      
4. Los valores fuera de estos límites se consideran atípicos. Podemos eliminarlos o transformarlos.
   
   Ventajas:
    - Fácil de interpretar.
    - No asume que los datos siguen una distribución normal.
    - Es ideal para datos con distribuciones sesgadas.

    Desventajas:
    - Puede eliminar datos válidos si la distribución es naturalmente dispersa.
    - No es adecuado para detectar atípicos en distribuciones normales con grandes variaciones.

In [ ]:
# Seleccionamos las columnas numéricas
numeric_columns = df_wine_clean.select_dtypes(include=['float64', 'int64'])

# Calcular Q1 y Q3
Q1 = numeric_columns.quantile(0.25)
Q3 = numeric_columns.quantile(0.75)

# Calcular IQR
IQR = Q3 - Q1

# Calcular límites inferior y superior
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
# Crear un nuevo DataFrame con estas estadísticas
iqr_summary = pd.DataFrame({
    "Q1 (25%)": Q1,
    "Q3 (75%)": Q3,
    "IQR": IQR,
    "Limite_Inferior": lower_bound,
    "Limite_Superior": upper_bound
})

#from IPython.display import display
#display(iqr_summary)
iqr_summary

In [ ]:
import pandas as pd

# Función para filtrar outliers usando IQR
def filtrar_outliers_iqr(df, iqr_summary):
    """
    Filtra los valores atípicos (outliers) de un DataFrame según los límites IQR proporcionados.

    Parámetros:
    - df: DataFrame con los valores numéricos.
    - iqr_summary: DataFrame con los límites inferior y superior para cada columna.

    Retorna:
    - DataFrame sin outliers.
    """
    return (
        df.copy()  # Copiar para evitar modificar el original
        .apply(lambda col: col.where(
            (col >= iqr_summary.loc[col.name, "Limite_Inferior"]) & 
            (col <= iqr_summary.loc[col.name, "Limite_Superior"])
        ))
        .dropna()  # Eliminar filas con valores NaN generados por la filtración
    )

# Aplicar la función de filtrado
df_wine_filtered = filtrar_outliers_iqr(numeric_columns, iqr_summary)

# Mostrar resultado
df_wine_filtered


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Definir tamaño de figura y DPI alto
plt.figure(figsize=(12, 6), dpi=150)

# Crear el boxplot con mejoras
sns.boxplot(data=df_wine_filtered, orient='h', width=0.7)

# Personalizar estilo y ejes
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel("Valores", fontsize=14)
plt.ylabel("Variables", fontsize=14)
plt.title("Distribución de las Variables en el Dataset", fontsize=16, fontweight="bold")

# Mostrar la gráfica
plt.show()

### Conclusión sobre el Filtrado de Outliers: 
#### Análisis del Filtrado con IQR

El uso del método IQR (Interquartile Range) para eliminar valores atípicos nos ha llevado a reducir el número total de registros en el dataset de 6497 a 4840, lo que representa una reducción cercana al 25% de los datos.

`Reflexión`:
Dado que el vino es un producto natural sujeto a variaciones en su proceso de fabricación, la presencia de valores extremos no necesariamente indica errores en la medición o registros incorrectos. Estas variaciones pueden ser causadas por múltiples factores en el proceso de fermentación, como:

Diferencias en la composición de la uva (según la región, clima y cosecha).
- Condiciones de fermentación (temperatura, tiempo, tipo de levaduras).
- Interacción de compuestos químicos, como la variabilidad en los sulfitos y la acidez volátil,
- Que dependen del control de oxígeno y la calidad del mosto.
  
El método de Six Sigma (±6σ) es aún más restrictivo que IQR, lo que significa que se eliminarían aún más datos en caso de aplicarlo. Sin embargo, decidimos no eliminar ningún dato con este método, ya que no encontramos indicios claros de errores en la medición.

Por lo tanto, eliminar un cuarto de los datos bajo el criterio del IQR podría falsear la distribución real del dataset y eliminar información valiosa sobre las características del vino.

## Análisis de la variable "quality" respecto a las demás variables
En esta sección, analizaremos cómo la variable objetivo "quality" (calidad del vino) se relaciona con las demás variables del dataset.
El objetivo de esta exploración es identificar patrones y tendencias que puedan ayudarnos a comprender qué características influyen en la calidad del vino.

In [ ]:
for col in df_wine_quality.select_dtypes(include=['float64']).columns:
    plt.figure(figsize=(8,4))
    sns.boxplot(x=df_wine_quality["quality"], y=df_wine_quality[col])
    plt.title(f'{col} vs Calidad del Vino')
    plt.show()

#### Conclusiones del Análisis de Boxplots
Tras analizar la relación entre la calidad del vino y sus variables fisicoquímicas, se han identificado los siguientes patrones:

**1️. Valores atípicos en todas las calidades**  
- La presencia de valores atípicos en todos los niveles de calidad indica que **no se trata de errores de medición**, sino de variabilidad inherente al proceso de producción del vino.

**2️. Acidez fija y calidad**  
- Se observa una tendencia **decreciente** en la acidez fija a medida que aumenta la calidad del vino.  
- Esto sugiere que vinos más ácidos podrían ser percibidos como de **menor calidad**.

**3️. Acidez volátil y calidad** 
- Los vinos con **mayor acidez volátil** tienden a ser de menor calidad.  
- Un exceso de acidez volátil está relacionado con **defectos sensoriales**, afectando negativamente la percepción del vino.

**4️. Azúcar residual**  
- La mayoría de los vinos presentan niveles **bajos** de azúcar residual, aunque existen valores atípicos extremadamente altos.  
- Esto podría deberse a **fermentaciones incompletas** o, en casos aislados, a errores de medición.  
- Los vinos de mayor calidad tienen un **control más estable del azúcar residual**, mientras que algunos vinos de menor calidad muestran valores desproporcionadamente altos.

**5️. Cloruros**  
- **Altos niveles de cloruros** se asocian con vinos de menor calidad.  
- En vinos de calidad superior, los valores están más concentrados y sin valores extremos.  
- Esto sugiere que un exceso de cloruros **afecta negativamente la percepción de calidad**.

**6️. Dióxido de azufre (libre y total)**  
- **No existe una relación clara** entre altos niveles de dióxido de azufre y la calidad del vino.  
- Sin embargo, los valores atípicos altos están más presentes en vinos de **menor calidad**, lo que sugiere que un exceso de SO₂ podría no estar asociado con vinos de alta gama.

**7️. Sulfatos** 
- **Los valores atípicos en sulfatos son más frecuentes en vinos de menor calidad**, lo que indica que niveles elevados podrían estar relacionados con un menor puntaje de calidad.  
- En vinos de calidad alta, la dispersión de sulfatos es menor, sugiriendo un **mejor equilibrio en su composición química**.

**8️. Alcohol y calidad**
- Los vinos de mejor calidad **tienden a tener un mayor contenido de alcohol**.  
- Además, los valores atípicos altos en alcohol suelen estar en vinos de calidad superior, lo que sugiere que un mayor grado alcohólico podría ser un **indicador de una mejor percepción sensorial**.


In [ ]:
#Conclusión
import pandas as pd

# Crear el DataFrame
data = {
    "Variable": [
        "Fixed Acidity", "Volatile Acidity", "Citric Acid", "Residual Sugar", "Chlorides",
        "Free Sulfur Dioxide", "Total Sulfur Dioxide", "Density", "pH", "Sulphates", "Alcohol"
    ],
    "Relación con calidad": [
        "No clara", "Vinos de baja calidad tienen valores altos", "No clara", "Valores altos en vinos de baja calidad",
        "Vinos de baja calidad tienen valores altos", "Valores altos en vinos de baja calidad",
        "Valores altos en vinos de baja calidad", "No clara", "No clara", "Valores altos en vinos de baja calidad",
        "Vinos de alta calidad tienen más alcohol"
    ],
    "¿Eliminar atípicos?": [
        "No", "Sí", "Sí", "Sí", "Sí", "Sí", "Sí", "No", "No", "Sí", "Sí"
    ],
    "Tipo de atípicos a eliminar": [
        "-", "Solo valores atípicos altos", "Solo valores atípicos altos", "Solo valores atípicos altos",
        "Solo valores atípicos altos", "Solo valores atípicos altos", "Solo valores atípicos altos", "-", "-",
        "Solo valores atípicos altos", "Solo valores atípicos bajos"
    ]
}

df = pd.DataFrame(data)
df


Tras analizar los boxplots de cada variable en relación con la calidad del vino, observamos que algunas variables presentan diferencias en la mediana según la calidad, lo que sugiere una posible relación entre ellas. Sin embargo, la visualización de los boxplots no nos permite cuantificar con precisión la fuerza de estas relaciones.

Por ello, utilizamos la correlación de `Pearson` como un primer enfoque para medir la relación lineal entre las variables numéricas y la calidad del vino. Aunque sabemos que Pearson asume normalidad y puede verse afectado por valores atípicos, nos permitirá identificar qué variables pueden tener una relación más fuerte con la calidad del vino.

Tras calcular Pearson, si encontramos variables con baja correlación (< 0.3), consideraremos otros métodos más robustos como `Spearman` o `Kendall`, que no requieren normalidad y son menos sensibles a valores extremos.

In [ ]:
import pandas as pd
df_wine_quality_2 = df_wine_quality.copy()

df_wine_quality_2 = df_wine_clean.select_dtypes(include=['float64', 'int64'])

# Calcular la correlación de Pearson
correlation_pearson = df_wine_quality_2.corr(method='pearson')

correlation_pearson


### ¿Por qué representamos la correlación con un mapa de calor?

El uso de un **heatmap** o mapa de calor es una herramienta fundamental para analizar la relación entre variables en nuestro dataset. Nos permite identificar patrones y tendencias de forma visual, facilitando la interpretación de la matriz de correlación de Pearson.

#### Beneficios del uso de un mapa de calor:

- **Visualización intuitiva:** Permite observar de un vistazo qué variables tienen una mayor o menor correlación con la calidad del vino.

- **Identificación rápida de patrones:** Los colores más intensos indican correlaciones más fuertes, ya sean **positivas** o **negativas**, ayudando a detectar relaciones clave entre las variables.

- **Comparación simultánea:** Facilita el análisis de cómo cada variable se relaciona con las demás, detectando posibles **multicolinealidades** (cuando dos o más variables están altamente correlacionadas entre sí).

- **Facilita la selección de variables:** En futuras fases del análisis, este enfoque nos permitirá **identificar las variables más relevantes** para predecir la calidad del vino, ayudando a optimizar el modelo de machine learning.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar el tamaño del gráfico
plt.figure(figsize=(12,8))

# Crear el heatmap
sns.heatmap(correlation_pearson, annot=True, cmap="coolwarm", fmt=".2f")

# Mostrar el gráfico
plt.title("Matriz de Correlación de Pearson")
plt.show()


### Análisis de la correlación de Pearson

Tras analizar los **boxplots** de la variable `quality` en relación con otras variables, procedemos a calcular la **matriz de correlación de Pearson**. Este análisis nos permite identificar **relaciones lineales** entre las características fisicoquímicas del vino y su calidad.

**Hallazgos clave**:
- **Alcohol (`0.44`)**: Es la variable con **mayor correlación positiva** con la calidad del vino, lo que sugiere que vinos con mayor contenido alcohólico tienden a ser mejor valorados.
- **Densidad (`-0.31`)**: Presenta una **correlación negativa moderada**, lo que indica que vinos con mayor densidad suelen tener menor calidad.
- **Volatile Acidity (`-0.27`)**: También muestra una **correlación negativa**, sugiriendo que vinos con mayor acidez volátil tienden a ser de menor calidad.
- **Sulphates (`0.19`)**: Exhibe una **relación positiva débil** con la calidad del vino.
- **Total Sulfur Dioxide (`-0.04`)**: Correlación prácticamente **nula**, indicando que no influye significativamente en la calidad del vino.

**Importante**: La correlación de Pearson solo mide relaciones **lineales** entre variables. Para capturar posibles relaciones **no lineales**, exploraremos la correlación de **Spearman** o **Kendall** en el siguiente paso.


In [ ]:
import pandas as pd

# Matriz de correlación de Spearman
spearman_corr = df_wine_quality_2.corr(method='spearman')
spearman_corr

# Matriz de correlación de Kendall
kendall_corr = df_wine_quality_2.corr(method='kendall')
kendall_corr


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))

# Para Spearman
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación de Spearman')
plt.show()

# Para Kendall
plt.figure(figsize=(10, 8))
sns.heatmap(kendall_corr, annot=True, cmap='coolwarm', center=0)
plt.title('Matriz de Correlación de Kendall')
plt.show()


#### ¿Qué significa esto?
Dado que ninguna variable individual tiene una relación clara con la calidad del vino, es posible que la calidad dependa de una combinación de múltiples factores en lugar de un solo parámetro químico. Esto sugiere que **técnicas más avanzadas** como modelos de `regresión múltiple` o métodos de `machine learning` podrían ser más útiles para predecir la calidad del vino.

## **Análisis de la Influencia de las Variables en la Calidad del Vino**

Hasta ahora, hemos evaluado la relación entre las variables fisicoquímicas y la calidad del vino mediante boxplots y la correlación de Pearson. Sin embargo, Pearson solo mide relaciones lineales y no nos permite determinar si existen diferencias significativas en la distribución de estas variables entre distintos niveles de calidad del vino.

Para abordar esta limitación, utilizaremos la **prueba de Kruskal-Wallis**, un test no paramétrico que nos permitirá evaluar si existen diferencias estadísticamente significativas en la distribución de cada variable para los distintos niveles de calidad del vino. A diferencia de ANOVA, Kruskal-Wallis no asume normalidad en los datos, lo que lo hace ideal para nuestro caso donde muchas variables no siguen una distribución normal.

### **Objetivo de la Prueba de Kruskal-Wallis**
- Determinar si las diferencias observadas en los boxplots son estadísticamente significativas.
- Evaluar qué variables presentan un impacto más fuerte en la calidad del vino.
- Filtrar aquellas variables con un valor p menor a 0.05, lo que indicaría que la variable tiene un efecto significativo en la calidad.

A continuación, implementamos la prueba de Kruskal-Wallis en nuestras variables numéricas.


In [ ]:
import scipy.stats as stats
import pandas as pd

# Suponiendo que tu DataFrame es df_wine_quality_2 y 'quality' es la variable categórica
variables_numericas = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 
                       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 
                       'density', 'pH', 'sulphates', 'alcohol']

# Diccionario para almacenar los resultados de Kruskal-Wallis
kruskal_results = {}

for var in variables_numericas:
    grupos = [df_wine_quality_2[var][df_wine_quality_2['quality'] == q] for q in df_wine_quality_2['quality'].unique()]
    stat, p_value = stats.kruskal(*grupos)
    kruskal_results[var] = {'Estadístico H': stat, 'p-value': p_value}

# Convertimos los resultados en un DataFrame para visualizar mejor
kruskal_df = pd.DataFrame(kruskal_results).T
kruskal_df

# Opcional: Filtrar variables con impacto significativo (p < 0.05)
significativas = kruskal_df[kruskal_df['p-value'] < 0.05]
print("\nVariables con impacto significativo en quality:")
significativas


### **Conclusión del test de Kruskal-Wallis**

El test de **Kruskal-Wallis** se realizó para analizar si existen diferencias significativas en la variable **quality** respecto a las demás variables numéricas del conjunto de datos. 

 **Interpretación de los resultados:**
- **Todas las variables analizadas tienen un impacto estadísticamente significativo en la calidad del vino (`p-value < 0.05`)**, lo que significa que hay diferencias en su distribución entre los distintos niveles de `quality`.
- **Alcohol, densidad, cloruros y acidez volátil tienen los valores de estadístico H más altos**, lo que indica que estas variables muestran las diferencias más marcadas en su distribución según la calidad del vino.

### **Puntos clave**
1. **El alcohol tiene la relación más fuerte con la calidad del vino (`H = 1397.32, p ≈ 0`):** Este resultado es consistente con la matriz de correlación de Pearson, donde el alcohol tenía la correlación más alta con `quality`.
2. **Densidad (`H = 758.89`) y cloruros (`H = 607.78`) también muestran una fuerte variabilidad entre los niveles de calidad.** La densidad puede estar relacionada con el contenido de azúcar residual y el alcohol, mientras que los cloruros pueden estar vinculados a la mineralidad del vino.
3. **El pH tiene un efecto menos pronunciado (`H = 15.27, p = 0.018`), lo que sugiere que, aunque hay una diferencia entre los niveles de calidad, su impacto es menor en comparación con otras variables.**
4. **El azúcar residual (`H = 39.17, p = 6.61e-07`) también muestra diferencias según la calidad, aunque en menor medida que las variables mencionadas anteriormente.**

### **Conclusión final**
 **El alcohol es la variable que más influye en la calidad del vino**, seguido por la **densidad, los cloruros y la acidez volátil**. La presencia de estas diferencias sugiere que estas características juegan un papel clave en la percepción de la calidad del vino y podrían ser utilizadas para modelar su clasificación.


## **Resumen del Proceso Hasta Ahora**

A lo largo del análisis, hemos aplicado diversas técnicas exploratorias y estadísticas para comprender cómo las variables fisicoquímicas influyen en la calidad del vino:

1. **Boxplots**: Visualizamos la distribución de cada variable en función de la calidad del vino, identificando patrones y la presencia de valores atípicos.
2. **Correlaciones (Pearson, Spearman, Kendall)**: Cuantificamos la relación entre la variable `quality` y las demás variables para detectar asociaciones lineales y no lineales.
3. **Prueba de Kruskal-Wallis**: Evaluamos si las distribuciones de las variables presentan diferencias significativas en función de la calidad del vino, sin asumir normalidad en los datos.

### **Resultados Clave**
- **Alcohol, densidad, cloruros y acidez volátil** son las variables que más influyen en la calidad del vino.  
- Los resultados obtenidos son consistentes a lo largo de las distintas metodologías utilizadas, reforzando la validez de nuestros hallazgos.  
- Al comprender estas relaciones, podremos seleccionar mejor las variables más relevantes para futuros modelos predictivos.  


Dado que estamos analizando un **dataset de vinos**, en el que las variables representan mediciones químicas, es importante considerar que los valores atípicos no necesariamente indican **errores de medición**, sino que pueden formar parte de la **variabilidad natural del proceso de fabricación del vino**.

**¿Por qué no asumimos errores de medida?**

En la producción de vino, la fermentación y otros procesos químicos pueden generar variaciones naturales en las mediciones.  
Muchos de los valores atípicos se encuentran en rangos de vinos de **calidad más baja**, lo que sugiere que podrían reflejar diferencias reales en la composición química de los vinos.  
Rechazar demasiados valores extremos podría **distorsionar el análisis** y llevarnos a conclusiones erróneas sobre la relación entre las variables y la calidad del vino.  

Dado que estos valores pueden afectar ciertos análisis y modelos predictivos, evaluaremos distintas estrategias para su manejo, asegurándonos de **minimizar el impacto sin perder información relevante**.  

### Aplicación de Winsorización para el tratamiento de valores atípicos

Dado que eliminar valores atípicos puede llevar a una pérdida significativa de información y distorsionar el análisis, optamos por una técnica más conservadora: **Winsorización**. 

La **Winsorización** es un método de tratamiento de outliers en el que en lugar de eliminar los valores extremos, estos se reemplazan por un percentil definido, reduciendo así su impacto sin afectar drásticamente la distribución de los datos.

#### ¿Por qué usamos Winsorización en lugar de eliminar outliers?
- **Preserva la cantidad total de datos**: No eliminamos observaciones, lo que evita sesgar el conjunto de datos.  
- **Menos impacto en la variabilidad natural**: Considerando que los valores extremos pueden reflejar procesos naturales en la producción del vino, esta técnica es más adecuada.  
- **Reduce el efecto de valores extremos en modelos predictivos**: Evita que los outliers dominen la relación entre variables sin necesidad de eliminarlos por completo.

Aplicaremos Winsorización con un umbral del **0.5% en ambos extremos**, lo que significa que los valores más bajos y más altos se reemplazarán por los valores del percentil 0.5 y 99.5 respectivamente.


In [ ]:
from scipy.stats.mstats import winsorize
import numpy as np

def aplicar_winsorizacion(df, limits=[0.005, 0.005]):

    df_winsorized = df.copy()  # Copiar para evitar modificar el original
    
    # Aplicar winsorización a cada columna numérica
    for col in df_winsorized.select_dtypes(include=['float64', 'int64']).columns:
        if col != "quality":  # Excluir la columna 'quality' si es la variable objetivo
            df_winsorized[col] = np.asarray(winsorize(df_winsorized[col].values, limits=limits))

    return df_winsorized

# Aplicar winsorización al DataFrame
df_wine_quality_winsorized = aplicar_winsorizacion(df_wine_quality_2)

# Reincorporamos la columna 'type' al DataFrame winsorizado
df_wine_quality_winsorized["type-wine"] = df_wine_quality["type-wine"]



In [ ]:
df_wine_quality_winsorized.sample(10)

In [ ]:
import pandas as pd

# Comparar antes y después de winsorización
stats_before = df_wine_quality_2.describe().T  # Estadísticas antes
stats_after = df_wine_quality_winsorized.describe().T  # Estadísticas después

# Unir ambas tablas para comparar mínimo y máximo
stats_comparison = pd.concat(
    [stats_before["min"], stats_before["max"], stats_after["min"], stats_after["max"]], axis=1
)
stats_comparison.columns = ["Min Antes", "Max Antes", "Min Después", "Max Después"]

display(stats_comparison)


## Conclusiones de la Winsorización

### Reducción de valores extremos  
- La winsorización ha reducido los valores más extremos de las variables.  
- **Ejemplo:** `residual sugar` tenía un máximo de **65.8** y ahora está en **19.4**, mientras que `total sulfur dioxide` bajó de **440** a **247**.  
- También se han ajustado los valores mínimos, aunque en menor medida.  

### Menor impacto en algunas variables  
- En variables como `pH`, `density` y `citric acid`, el efecto es mínimo debido a la ausencia de valores extremos muy marcados.  
- Esto indica que la winsorización ha tenido mayor impacto en variables con mayor dispersión y presencia de outliers.  

### Ajuste más controlado  
- A diferencia de eliminar outliers, la winsorización **mantiene todos los datos dentro de un rango razonable** sin eliminar filas del dataset.  
- Así, evitamos la pérdida de información valiosa y prevenimos distorsiones excesivas en la distribución.  

### Mejor comportamiento en modelos predictivos  
- Con una distribución más estable, el dataset será más interpretable y favorecerá el rendimiento de modelos de machine learning.  
- Variables como `volatile acidity` y `free sulfur dioxide` ya no tendrán valores extremos que podrían influir desproporcionadamente en el análisis.  


# 4. Almacenar los datos los datos limpios en SQLite

Después de procesar y limpiar los datos, almacenaremos el **DataFrame final** en una base de datos para facilitar su gestión y futuras consultas.  
Para este propósito, utilizaremos el dataset **`df_wine_quality_winsorized`**, que contiene los datos tras el tratamiento de valores atípicos mediante winsorización.  

---

## ¿Qué es SQLite y por qué lo usamos?

**SQLite** es un sistema de gestión de bases de datos ligero y sin servidor, ideal para proyectos de análisis de datos porque:  

- **Es fácil de usar:** No requiere configuración de un servidor, lo que facilita su implementación en notebooks y proyectos locales.  
- **Es eficiente y rápido:** Permite realizar consultas SQL sin necesidad de conectarse a una base de datos externa.  
- **Facilita la organización de datos:** Nos permite almacenar los datos limpios y acceder a ellos posteriormente sin necesidad de procesar el dataset desde cero.  
- **Compatible con Python:** La librería `sqlite3` nos permite interactuar con la base de datos directamente desde nuestro código.  

Al almacenar nuestros datos en SQLite, garantizamos una mejor **persistencia**, **estructura** y **accesibilidad** para futuras fases del análisis y modelado.

PAra cualquier consulta sigue la documentación oficial de SQLite: [SQLite Python Documentation](https://docs.python.org/3/library/sqlite3.html).

Para comenzar, necesitas conectar tu script de Python a una base de datos SQLite. Puedes hacerlo con:

In [ ]:
import sqlite3

# Conectar a una base de datos y se crea automáticamente ya que no existe
con = sqlite3.connect(r"C:\Users\Oscar\Documents\MBIT_Oscar\MBIT_202501_Proyecto_Consolidacion_1\data\bbdd_wine_quality.db")

# Crear un cursor para ejecutar comandos SQL
cur = con.cursor()

In [ ]:
# Intentar ejecutar una consulta simple para verificar la conexión
try:
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablas = cur.fetchall()
    print("Conexión exitosa. Tablas en la base de datos:", tablas)
except sqlite3.Error as e:
    print("Error en la conexión:", e)

Ahora que hemos configurado una conexión y un cursor, podemos crear una tabla con nuestros datos bien filtrados

In [ ]:
import pandas as pd

# Insertar el DataFrame en la base de datos SQLite
df_wine_quality_winsorized.to_sql("wine_quality_clean", con, if_exists="replace", index=False)

# Confirmar que la tabla se creó correctamente
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tablas en la base de datos:", cur.fetchall())


# 5. Realizaremos las siguientes consultas en SQLite

##### 5.1 Consulta 1: ¿Cuál es el promedio de calidad (`quality`) por tipo de vino (`type`)?

In [ ]:
# Ejecutar la consulta SQL para obtener el promedio de calidad por tipo de vino
query = """
SELECT "type-wine", AVG(quality) AS avg_quality
FROM wine_quality_clean
GROUP BY "type-wine";
"""

# Ejecutamos la consulta
cur.execute(query)

# Obtenemos los resultados
resultados = cur.fetchall()

# Mostramos los resultados
for row in resultados:
    print(f"Tipo de vino: {row[0]}, Calidad promedio: {row[1]:.2f}")


##### 5.2 Consulta 2: ¿Cuántos vinos tienen un nivel de alcohol superior a 10.5, agrupados por tipo?

In [ ]:
# ¿Cuántos vinos tienen un nivel de alcohol superior a 10.5, agrupados por tipo?
query = """
SELECT "type-wine", COUNT(*) AS count_vinos
FROM wine_quality_clean
WHERE alcohol > 10.5
GROUP BY "type-wine";
"""

cur.execute(query)
resultados = cur.fetchall()

# Mostrar resultados de forma legible
for row in resultados:
    print(f"Tipo de vino: {row[0]}, Cantidad de vinos con alcohol > 10.5: {row[1]}")


##### 5.3 Consulta 3: Obtén el conteo de vinos por nivel de acidez (`fixed acidity`) agrupados en rangos (por ejemplo, de 0-5, 5-10, 10-15).

In [ ]:
# 3. Consulta 3: Obtén el conteo de vinos por nivel de acidez (`fixed acidity`) agrupados en rangos (por ejemplo, de 0-5, 5-10, 10-15).
query = """
SELECT 
    CASE 
        WHEN "fixed acidity" >= 0 AND "fixed acidity" < 5 THEN '0-5'
        WHEN "fixed acidity" >= 5 AND "fixed acidity" < 10 THEN '5-10'
        WHEN "fixed acidity" >= 10 AND "fixed acidity" < 15 THEN '10-15'
        ELSE '15+' 
    END AS acidity_range,
    COUNT(*) AS count_vinos
FROM wine_quality_clean
GROUP BY acidity_range
ORDER BY acidity_range;

"""

cur.execute(query)
resultados = cur.fetchall()

# Mostrar resultados de forma legible
for row in resultados:
    print(f"Rango de acidez: {row[0]}, Cantidad de vinos: {row[1]}")


# 6. Exportación de los datos a JSONLines
De cara a una potencial insercion en una base de datos noSQL como `mongoDB`, podemos servirnos de pandas para preparar los datos.

##### ¿Qué estructura de datos de python es la más similar a un documento noSQL?

MongoDB almacena datos en formato BSON (una versión binaria de JSON), donde cada documento es una estructura clave-valor, igual que los diccionarios en Python.

##### Usa Pandas para transformar los datos de una de las consultas en un archivo JSONLines.

In [ ]:
#df_wine_quality_winsorized.head()
df_wine_quality_winsorized.to_json(
    path_or_buf = r"C:\Users\Oscar\Documents\MBIT_Oscar\MBIT_202501_Proyecto_Consolidacion_1\data\mongodb_wine_quality.jsonl",
    orient = "records",
    lines = True,
    index = False
)

##### Usa la librería `jsonlines` para guardar el archivo.

In [ ]:
import jsonlines

with jsonlines.open("../data/wine_quality_jsonlines.jsonl", mode="w") as writer:
    writer.write_all(df_wine_quality_winsorized.to_dict(orient="records"))

##### ¿Qué problemas podrían surgir al transformar un dataframe en jsonlines?

 Tipos de datos incompatibles:

- `np.array`: Se convierte en listas, pero algunas bases NoSQL no lo soportan bien.
- `pd.datetime`: Puede que se transforme en string en un formato inesperado.
- `NaN en pandas`: En JSON se convierte en null, lo que puede causar problemas en bases de datos.

##### Añade una columna que sea originalmente un `np.array`,¿qué sucede al transformarlo en jsonlines?

In [ ]:
import numpy as np

# Agregar una columna con un np.array simple
df_wine_quality_winsorized["array_column"] = np.array([np.array([1, 2, 3])] * len(df_wine_quality_winsorized))

# Exportar a JSONLines
df_wine_quality_winsorized.to_json(
    "../data/wine_quality_with_array.jsonl", 
    orient="records", 
    lines=True
)


##### ¿Qué sucede aquí?

Durante el proceso de manipulación de datos, nos encontramos con un **error** al intentar agregar una columna con un array multidimensional.

##### Causa del problema:
- Intentamos agregar una columna que contiene un `np.array` con **forma (6497, 3)**.
- **Error esperado**: *Pandas no admite arrays de múltiples dimensiones en una sola columna* porque espera valores escalares o listas.
- Esto genera conflictos al intentar **exportar el DataFrame a formato JSONLines**, ya que `pandas.to_json()` no sabe cómo manejar esta estructura.

##### Posible solución:
Para evitar este problema, podemos **convertir el array en múltiples columnas separadas** o transformarlo en una lista con valores serializables antes de incorporarlo al DataFrame.

---
Ahora exploraremos opciones para resolver este inconveniente y garantizar la correcta exportación de los datos.


In [ ]:
df_wine_quality_winsorized["array_column"] = [np.array([1, 2, 3]).tolist()] * len(df_wine_quality_winsorized)

# Exportar a JSONLines
df_wine_quality_winsorized.to_json(
    "../data/wine_quality_with_array.jsonl", 
    orient="records", 
    lines=True
)

##### ¿Qué estamos haciendo aquí para corregirlo?

Hemos encontrado un problema al intentar almacenar un `np.array` multidimensional en un **DataFrame de pandas**. Para solucionarlo, aplicamos una conversión que permite su correcta manipulación y exportación.

##### Solución aplicada:
1️. **Convertimos el `np.array` a una lista de Python** usando `.tolist()`.  
   - En lugar de asignar un array directamente, ahora cada celda de la columna `"array_column"` contiene una **lista** con valores del tipo `[1, 2, 3]`, lo que sí es aceptado por `pandas` y formatos de exportación como JSONLines.

2️. **Verificamos compatibilidad con `pandas.to_json()`**  
   - Ahora, al usar `pandas.to_json(..., lines=True)`, **no hay errores**, y el archivo se genera correctamente.

---

##### Conclusión Final

**Limitación de `pandas`**  
- `np.array` **no puede asignarse directamente** a un DataFrame si es multidimensional.  
- `pandas` solo admite valores escalares o listas dentro de una celda.

**Conversión a lista**  
- **Al aplicar `.tolist()`, el problema desaparece**, permitiendo que `pandas` y JSONLines manejen correctamente los datos.  
- Algunas **bases de datos NoSQL** pueden aceptar listas directamente, pero otras pueden requerir que se guarden como **strings**.

**Consideraciones antes de exportar datos**  
- **Asegurar compatibilidad** con el formato de destino antes de exportar.  
- Revisar si la estructura de datos es soportada por JSON, SQL o bases NoSQL.  

**Lección clave:**  
Antes de exportar datos a **JSONLines** o bases de datos **NoSQL**, es fundamental **verificar la compatibilidad** de los tipos de datos con el formato final para evitar errores en la manipulación y análisis de la información.


##### Añade una columna que sea originalmente un `pd.datetime`,¿qué sucede al transformarlo en jsonlines?

In [ ]:
import pandas as pd

df_wine_quality_winsorized["date_column"] = pd.to_datetime("2024-01-01")

df_wine_quality_winsorized.to_json(
    "../data/wine_quality_with_date.jsonl", 
    orient="records", 
    lines=True
)

In [ ]:
with open("../data/wine_quality_with_date.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(line.strip())  # Muestra la línea completa en formato JSON
        if i == 2:  # Solo mostramos las primeras 2 líneas
            break

##### Conversión de `pd.datetime` en JSONLines

Al exportar el **DataFrame** a **JSONLines**, hemos observado que la columna `"date_column"` no mantiene su formato original y en su lugar aparece como un número entero grande.

##### ¿Por qué sucede esto?

**Serialización de `pd.datetime` en JSONLines**
- `pandas`, al exportar columnas de tipo `datetime`, **convierte automáticamente** los valores a **timestamps UNIX** en milisegundos.
- JSONLines **no tiene un tipo de dato nativo para fechas**, por lo que almacena estos valores como **números enteros** representando el número de milisegundos desde **1970-01-01 (Epoch Time)**.

In [ ]:
df_wine_quality_winsorized["date_column"] = pd.to_datetime("2024-01-01")

df_wine_quality_winsorized["date_column"] = df_wine_quality_winsorized["date_column"].astype(str)

df_wine_quality_winsorized.to_json(
    "../data/wine_quality_with_date.jsonl", 
    orient="records", 
    lines=True
)

with open("../data/wine_quality_with_date.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        print(line.strip())  # Muestra la línea completa en formato JSON
        if i == 2:  # Solo mostramos las primeras 5 líneas
            break


Utilizando es astype(str) En lugar de guardar la fecha como 1704067200000 (milisegundos desde Epoch), ahora se almacena en formato ISO 8601 ("2024-01-01 00:00:00"), haciéndolo más legible y compatible con bases de datos NoSQL como MongoDB.

# 7. Análisis de la calidad del vino

##### 7.1 Inspecciona qué caracteriza a los vinos tintos y blancos con mayor calidad (`quality`).

##### 7.2 Usa análisis estadístico, gráficos o cualquier técnica que consideres relevante para identificar patrones.

### Objetivo del Análisis de la Calidad del Vino

En esta sección, nuestro objetivo es identificar patrones en los datos que nos ayuden a comprender qué factores influyen en la calidad del vino. Para ello, utilizaremos visualizaciones y modelos estadísticos para evaluar las relaciones entre las características químicas y la variable `quality`.

El análisis se enfocará en dos aspectos principales:

1️. **Identificación de Patrones**:  
   - Utilizaremos gráficos para visualizar cómo las diferentes variables se relacionan con la calidad del vino.  
   - Analizaremos tendencias y posibles agrupaciones dentro de los datos.

2️. **Modelización Predictiva**:  
   - Aplicaremos técnicas de aprendizaje automático para intentar predecir la calidad del vino en función de sus características.  
   - Compararemos diferentes modelos para evaluar cuál ofrece un mejor desempeño en la predicción de `quality` utilizando el dataset original.

Este enfoque nos permitirá no solo entender qué factores están asociados con la calidad del vino, sino también desarrollar herramientas para predecir su clasificación con base en sus propiedades químicas.


# Elección de Modelos para Identificar Patrones en la Calidad del Vino

Para responder al ejercicio y determinar qué características definen a los vinos de mayor calidad, debemos identificar patrones en los datos. Esto implica segmentar los vinos en grupos y analizar relaciones entre variables. Aquí es donde los modelos de **clustering** y **análisis de correlación** resultan útiles.

## ¿Qué modelo utilizar?

Para este caso, **clustering** es la mejor opción, ya que nos permite descubrir grupos ocultos en los datos sin necesidad de etiquetas previas. Esto nos ayudará a **identificar qué características diferencian los vinos de mejor calidad**.

---

## Paso 1: Análisis Exploratorio Inicial (EDA)  

Antes de aplicar clustering, es fundamental realizar un **análisis exploratorio de datos (EDA)** para comprender la distribución de `quality` y su relación con otras variables.

### 1. Inspeccionar la Distribución de `quality`

- **Objetivo:** Analizar cuántos vinos hay en cada nivel de calidad.  
- **Cómo hacerlo:** Utilizar un gráfico de barras o `value_counts()` para visualizar la frecuencia de cada categoría de calidad.

### ¿Qué buscamos?  

- Evaluar si la calidad está **bien distribuida** o si hay una concentración excesiva en ciertos valores (por ejemplo, si la mayoría de los vinos tienen calidad **5-6**).  
- Detectar **clases raras**, como muy pocos vinos con calidad **9**, lo que podría influir en los resultados del clustering.  

---

A continuación, procederemos con la inspección gráfica y estadística de la variable `quality` antes de aplicar técnicas de clustering.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Conteo de vinos por calidad
plt.figure(figsize=(8,5))
sns.countplot(x=df_wine_quality_winsorized['quality'], hue=df_wine_quality_winsorized['quality'], palette='viridis', legend=False)
plt.title("Distribución de la Calidad del Vino")
plt.xlabel("Calidad")
plt.ylabel("Número de vinos")
plt.show()

# Ver valores únicos y su conteo
df_wine_quality_winsorized["quality"].value_counts()


**2️. Revisar Correlaciones con quality**
- Objetivo: Identificar qué variables podrían influir más en la calidad.
- Cómo hacerlo: Matriz de correlación con quality.
**¿Qué buscamos?**

Variables con alta correlación positiva o negativa con quality.
Por ejemplo, si alcohol tiene una correlación fuerte con quality, podría ser un factor clave en vinos de alta calidad.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

# Filtrar solo las columnas numéricas antes de calcular la correlación
numeric_cols = df_wine_quality_winsorized.select_dtypes(include=['float64', 'int64'])
sns.heatmap(numeric_cols.corr(), annot=True, cmap="coolwarm", fmt=".2f")

plt.title("Matriz de Correlación")
plt.show()


**3. Revisar Distribución de Variables Clave**
- Objetivo: Ver cómo están distribuidas variables importantes como alcohol, density, volatile acidity, chlorides.
- Cómo hacerlo: Histogramas y Boxplots.

**¿Qué buscamos?**

Si hay diferencias en las distribuciones entre vinos de distinta calidad.
Si hay valores extremos en alguna variable.
Si alguna variable sigue una distribución normal o está sesgada.

In [ ]:
features = ["alcohol", "density", "volatile acidity", "chlorides"]

plt.figure(figsize=(12, 8))
for i, col in enumerate(features):
    plt.subplot(2, 2, i+1)
    sns.histplot(df_wine_quality_winsorized[col], kde=True, bins=30, color="teal")
    plt.title(f"Distribución de {col}")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
for i, col in enumerate(features):
    plt.subplot(2, 2, i+1)
    sns.boxplot(x=df_wine_quality_winsorized["quality"], y=df_wine_quality_winsorized[col])
    plt.title(f"{col} vs Calidad del Vino")
plt.tight_layout()
plt.show()


**4. Normalizar Variables Antes del Clustering**
- Objetivo: Escalar todas las variables a una misma escala para evitar sesgos en los modelos de clustering.
- Cómo hacerlo: Usar StandardScaler o MinMaxScaler.
  
**¿Qué buscamos?**

- Que todas las variables estén en la misma escala antes de hacer clustering.
- Que valores extremos no afecten demasiado el análisis.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Excluir tanto "quality" como "type-wine" antes de escalar
scaled_features = scaler.fit_transform(df_wine_quality_winsorized.drop(columns=["quality", "type-wine"]))

# Convertimos a DataFrame manteniendo nombres originales y reiniciamos el índice
df_wine_scaled = pd.DataFrame(scaled_features, columns=df_wine_quality_winsorized.drop(columns=["quality", "type-wine"]).columns)
df_wine_scaled = df_wine_scaled.reset_index(drop=True)

# Reincorporar las columnas "quality" y "type-wine" sin modificaciones
df_wine_scaled["quality"] = df_wine_quality_winsorized["quality"].reset_index(drop=True)
df_wine_scaled["type-wine"] = df_wine_quality_winsorized["type-wine"].reset_index(drop=True)

df_wine_scaled.head()


# Conclusiones del Análisis Exploratorio

## Distribución de la Calidad del Vino

- La mayoría de los vinos tienen una calidad entre **5 y 6**, lo que indica que el dataset está sesgado hacia valores intermedios.
- Existen muy pocos vinos con calidad **3 y 9**, lo que puede generar problemas si se quieren construir modelos de predicción, ya que estas clases están **desbalanceadas**.

---

## Matriz de Correlación

- **Alcohol** es la variable con mayor correlación positiva con la calidad del vino (~0.44), lo que sugiere que vinos con mayor contenido de **alcohol** tienden a tener mejor calidad.
- **Densidad** y **acidez volátil** tienen correlaciones negativas moderadas con la calidad del vino. Esto sugiere que vinos con menor **densidad** y menor **acidez volátil** tienden a ser de mejor calidad.
- **Cloruros** también presentan una correlación negativa con la calidad, lo que indica que **altos niveles de cloruros** podrían estar relacionados con vinos de menor calidad.
- Otras variables tienen correlaciones más débiles con la calidad.

---

## Distribución de Variables Clave

- **Alcohol:** Distribución sesgada a la derecha, con la mayoría de los valores entre **9 y 12.5**.
- **Densidad:** Tiene una distribución centrada en torno a **0.995**, lo que sugiere que la variabilidad en la densidad no es tan alta.
- **Acidez volátil:** Distribución sesgada a la derecha, con algunos valores extremos.
- **Cloruros:** También muestra una distribución sesgada con algunos valores atípicos.

---

## Boxplots de `quality` vs Variables Clave

- Se observa un **incremento** en el contenido de **alcohol** en los vinos de mayor calidad.
- La **densidad** parece **disminuir** en vinos de mayor calidad.
- La **acidez volátil** muestra una relación **inversa** con la calidad.
- El contenido de **cloruros** también es **menor** en vinos de mejor calidad.

---

## Estandarización de Datos

- Se aplicó **`StandardScaler`** para **normalizar** las variables antes de aplicar clustering.
- Ahora todas las variables tienen una **media de 0** y **desviación estándar de 1**, lo que es **esencial** para evitar sesgos en los modelos.
---

## Aplicación de Clustering para Identificar Patrones en la Calidad del Vino

Después de haber explorado la distribución de las variables y analizado sus relaciones con la calidad del vino, pasamos a aplicar **técnicas de clustering**. Estas técnicas nos permitirán agrupar los vinos según sus características químicas y ver si los vinos de mayor calidad comparten patrones comunes.

### ¿Por qué usamos clustering?
- **No tenemos etiquetas explícitas**: Aunque tenemos una variable de calidad, queremos descubrir patrones en las características sin imponer una segmentación previa.
- **Ayuda a descubrir grupos naturales**: Nos permite identificar segmentos de vinos con propiedades similares, lo que puede ayudar a comprender qué define un vino de mayor calidad.
- **Puede mejorar modelos predictivos**: Si encontramos grupos bien diferenciados, esto podría servir como una nueva característica para futuros modelos de clasificación.

### ¿Qué técnica usaremos?
Para este análisis, aplicaremos **K-Means**, una de las técnicas más utilizadas en clustering. Usaremos el **método del codo** para determinar el número óptimo de clusters y posteriormente evaluaremos la separación de los grupos utilizando la métrica de **silhouette score**.



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from mpl_toolkits.mplot3d import Axes3D  # Para visualización en 3D

In [ ]:
# Definir un rango de posibles clusters
k_values = range(2, 11)  # Probamos de 2 a 10 clusters
inertia_values = []  # Aquí guardaremos la inercia para cada k

# Aplicamos K-Means para cada valor de k
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(df_wine_scaled.drop(columns=['quality', 'type-wine']))  # Aplicamos sobre datos sin la columna "quality"
    inertia_values.append(kmeans.inertia_)  # Guardamos la inercia

# Visualizamos el método del codo
plt.figure(figsize=(8, 5))
plt.plot(k_values, inertia_values, marker="o", linestyle="--", color="b")
plt.xlabel("Número de clusters (k)")
plt.ylabel("Inercia")
plt.title("Método del Codo para K-Means")
plt.grid(True)
plt.show()

In [ ]:
# Elegimos k óptimo basado en la gráfica anterior (suponiendo k=3)
optimal_k = 4  # Ajustar este valor según el método del codo

# Aplicamos K-Means
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df_wine_scaled["cluster"] = kmeans.fit_predict(df_wine_scaled.drop(columns=["quality", 'type-wine']))  # Guardamos los clusters en el DataFrame

# Visualizamos el tamaño de cada cluster
df_wine_scaled["cluster"].value_counts()


In [ ]:
# Calcular el coeficiente de silueta
silhouette_avg = silhouette_score(df_wine_scaled.drop(columns=["quality", "cluster",'type-wine']), df_wine_scaled["cluster"])
print(f"Coeficiente de silueta para k={optimal_k}: {silhouette_avg:.3f}")

In [ ]:
# Seleccionamos tres variables importantes para la visualización
features_3d = ["alcohol", "density", "volatile acidity"]

# Creamos la figura 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection="3d")

# Scatter plot en 3D con colores según los clusters
scatter = ax.scatter(df_wine_scaled[features_3d[0]], 
                     df_wine_scaled[features_3d[1]], 
                     df_wine_scaled[features_3d[2]], 
                     c=df_wine_scaled["cluster"], cmap="viridis", s=50)

# Etiquetas de los ejes
ax.set_xlabel(features_3d[0])
ax.set_ylabel(features_3d[1])
ax.set_zlabel(features_3d[2])
ax.set_title(f"Visualización 3D de Clusters con K-Means (k={optimal_k})")

# Agregar barra de colores
plt.colorbar(scatter, ax=ax)
plt.show()

## ¿Por qué no elegimos K-Means?

Tras aplicar el **método del codo**, determinamos que **k = 4** era un valor óptimo para segmentar los vinos. Sin embargo, al calcular el **coeficiente de silueta** (*0.25*), observamos que los clusters no estaban bien separados y presentaban una superposición significativa.

### Limitaciones de K-Means en este caso:
- **Baja separación de clusters**: Un coeficiente de silueta bajo indica que los grupos encontrados no son claramente distintos.
- **Distribución difusa en 3D**: La visualización tridimensional mostró que los clusters no tienen fronteras bien definidas.
- **Limitaciones de la distancia euclidiana**: K-Means asume que los clusters son esféricos y equidistantes, lo cual no parece ser el caso en nuestros datos.

### Próximos pasos: Exploración de alternativas
Para mejorar la segmentación, exploraremos técnicas más avanzadas como:
- **PCA (Análisis de Componentes Principales)**: Para reducir la dimensionalidad y visualizar mejor la estructura de los datos.  
- **DBSCAN**: Un algoritmo basado en densidad que puede detectar mejor grupos de diferentes formas y tamaños, lo que podría ser más adecuado para identificar patrones ocultos en los vinos.  

---

## ¿Por qué usamos PCA y cuál es el objetivo?

El **Análisis de Componentes Principales (PCA)** es una técnica de reducción de dimensionalidad que nos permite transformar un conjunto de variables correlacionadas en un nuevo conjunto de variables **no correlacionadas**, llamadas **componentes principales**.

### ¿Por qué aplicamos PCA en nuestro análisis?
- **Reducción de dimensionalidad**: Nuestros datos contienen múltiples variables químicas del vino, y PCA nos ayuda a reducir su número sin perder información clave.  
- **Eliminación de redundancia**: Muchas variables están correlacionadas, y PCA permite capturar la mayor varianza en menos componentes.  
- **Visualización de los datos**: Al reducir los datos a **dos componentes principales**, podemos graficarlos en 2D y analizar su estructura de manera más intuitiva.  

### Objetivo del PCA en este estudio:
- Convertir las múltiples variables en **dos componentes principales** para facilitar la interpretación.  
- **Identificar patrones y separaciones** entre los vinos según sus características.  
- Facilitar la aplicación de algoritmos de clustering, asegurando que las dimensiones relevantes sean utilizadas de manera eficiente.  


In [ ]:
# Importar librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Definir el número de componentes principales
n_components = 2  # Queremos reducir los datos a 2 dimensiones

# Aplicamos PCA
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(df_wine_scaled.drop(columns=["quality", "cluster", 'type-wine']))

# Convertimos a DataFrame para visualizar
df_pca = pd.DataFrame(principal_components, columns=[f"PC{i+1}" for i in range(n_components)])
df_pca["cluster"] = df_wine_scaled["cluster"]  # Agregamos la columna de clusters para ver su distribución

# Mostramos las primeras filas
df_pca.head()

In [ ]:
# Visualización en 2D de los componentes principales
plt.figure(figsize=(10, 6))
scatter = plt.scatter(df_pca["PC1"], df_pca["PC2"], c=df_pca["cluster"], cmap="viridis", alpha=0.6)
plt.colorbar(scatter, label="Cluster")
plt.xlabel("Primer Componente Principal (PC1)")
plt.ylabel("Segundo Componente Principal (PC2)")
plt.title("Visualización de los vinos en el espacio PCA")
plt.show()

In [ ]:
# Varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_

# Mostrar la varianza explicada
for i, var in enumerate(explained_variance):
    print(f"PC{i+1}: {var:.4f} ({var*100:.2f}%)")

## Conclusiones del análisis con PCA

El **Análisis de Componentes Principales (PCA)** logró reducir la dimensionalidad de los datos, **conservando más del 50% de la varianza total**, distribuyéndose de la siguiente manera:

- **Primer componente principal (PC1):** Explica el **28.35%** de la varianza.
- **Segundo componente principal (PC2):** Explica el **22.97%** de la varianza.

### ¿Qué nos indica la visualización en el espacio PCA?
- **Patrones diferenciados:** Se observa que los vinos tienden a agruparse en diferentes zonas, lo que sugiere que sus características químicas presentan cierta estructura subyacente.  
- **Separación parcial:** Aunque se identifican agrupaciones, **algunos clusters no están completamente diferenciados**, lo que indica que las variables originales aún contienen información relevante.  
- **Posible necesidad de más componentes:** Si bien PCA ha permitido visualizar mejor la relación entre los vinos, puede ser necesario considerar **más componentes** para mejorar la representación y separación de los grupos.  

### ¿Por qué es útil este análisis?
Este análisis nos ayuda a explorar la **estructura oculta en los datos** y evaluar si el **clustering aplicado** es realmente adecuado para segmentar la calidad del vino. A partir de esta reducción de dimensionalidad, podemos mejorar los algoritmos de clustering y validar su desempeño en la clasificación de los vinos. 

---

### Construcción y Evaluación del Modelo Predictivo
En los análisis previos, exploramos los datos mediante clustering y PCA con el objetivo de identificar patrones ocultos en la calidad del vino. Sin embargo, los resultados indicaron que la segmentación mediante clustering no proporcionaba una separación clara entre vinos de distintas calidades. Por ello, optamos por un modelo supervisado que permita predecir directamente la variable quality a partir de sus características químicas.



#### Regresión Lineal

La regresión lineal es un método estadístico y de Machine Learning que busca modelar la relación entre una variable objetivo (o dependiente) $y$ y una o más variables predictoras (o independientes) $X_1, X_2, \dots, X_n$. El modelo asume que la variable objetivo puede expresarse como una combinación lineal de los predictores, más un término de error:

$$
y \approx \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_n X_n
$$

##### 1. ¿Qué es hacer una regresión lineal?
- **Objetivo:** Encontrar los coeficientes $\beta_0, \beta_1, \dots, \beta_n$ que mejor expliquen la relación entre los predictores y la variable objetivo.
- **Método de ajuste:** Se ajusta el modelo minimizando la suma de los errores cuadrados (a través del método de Mínimos Cuadrados Ordinarios) u otra función de costo.

##### 2. ¿Cómo se obtiene?
- **Resolución de ecuaciones:** En la forma clásica, se obtienen los coeficientes resolviendo las ecuaciones que minimizan la suma de los cuadrados de los residuos (la diferencia entre los valores reales y los valores predichos).
- **Implementación práctica:** Utilizando bibliotecas de Machine Learning (por ejemplo, `scikit-learn` en Python), el modelo se ajusta mediante métodos numéricos que calculan los coeficientes $\beta$.
- **Manejo de múltiples variables:** En el caso de múltiples variables, se pueden utilizar técnicas de regularización (como Ridge o Lasso) para controlar el sobreajuste y la multicolinealidad.

##### 3. ¿Qué se espera obtener?
- **Modelo interpretativo:** Permite entender cómo cada variable independiente influye en la variable objetivo. Cada coeficiente $\beta_i$ indica el cambio en $y$ ante un cambio unitario en $X_i$.
- **Predicciones para nuevos datos:** Una vez ajustado, el modelo puede utilizarse para predecir el valor de $y$ al introducir nuevos valores de $X_1, X_2, \dots, X_n$.
- **Evaluación del desempeño:** Se utilizan métricas como RMSE (Error Cuadrático Medio) y MAE (Error Absoluto Medio) para medir la precisión del modelo y su capacidad de generalización.

### Aplicación en la Calidad del Vino
En el contexto de la calidad del vino, la regresión lineal se utiliza para:
- Relacionar las características fisicoquímicas (por ejemplo, acidez, pH, alcohol, etc.) con la calificación de calidad ($quality$).
- Identificar cuáles variables tienen mayor impacto en la calidad del vino.
- Estimar el valor de $quality$ para nuevos vinos, permitiendo predecir la calidad con un modelo simple y fácilmente interpretativo.


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Definir X e y para la regresión lineal
X = df_encoded.drop(columns=["quality", 'type-wine']) 
y = df_encoded["quality"]  

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    shuffle=True
)

# Crear y entrenar el modelo de regresión lineal
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error
#Evaluamos el modelo para ver como de bien predice la calidad del vino
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")


In [ ]:
#Evaluamos la estabilidad del modelo
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

In [ ]:
# valores cercanos a 1 es un buen ajuste, cercano a 0 es negativo
r2 = model.score(X_test, y_test)
print(f"R² Score: {r2:.2f}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red", linestyle="dashed")  # Línea de referencia ideal
plt.xlabel("Calidad Real")
plt.ylabel("Calidad Predicha")
plt.title("Comparación de Predicciones vs. Valores Reales")
plt.show()

In [ ]:
#entendemos cómo cada variable afecta la predicción de la calidad del vino:
coefficients = pd.DataFrame(model.coef_, X_train.columns, columns=['Coeficiente'])
coefficients = coefficients.sort_values(by="Coeficiente", ascending=False)
print(coefficients)


### Conclusiones

---

## Regresión Logística

La **Regresión Logística** es un modelo estadístico utilizado para predecir variables categóricas, especialmente cuando la variable objetivo tiene solo dos posibles valores (**clasificación binaria**). En este caso, queremos clasificar los vinos en dos categorías:

- **Alta calidad** (`quality > 7`, representado como `1`).
- **Baja calidad** (`quality ≤ 7`, representado como `0`).

A diferencia de la **Regresión Lineal**, que predice valores continuos, la **Regresión Logística** estima la **probabilidad** de que una observación pertenezca a una clase específica. Para ello, en lugar de una función lineal simple, la regresión logística aplica la función **sigmoide** para convertir las predicciones en probabilidades entre `0` y `1`.

### **1. Fórmula Matemática de la Regresión Logística**
La relación entre las variables predictoras y la variable objetivo se modela con la siguiente ecuación:

$$
y \approx \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_n X_n
$$

Sin embargo, como queremos predecir probabilidades en lugar de valores continuos, aplicamos la función sigmoide \( \sigma(z) \):

$$
P(y=1 | X) = \sigma(\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_n X_n)
$$

donde la **función sigmoide** es:

$$
\sigma(z) = \frac{1}{1 + e^{-z}}
$$

Esto asegura que la salida siempre esté en el rango \( [0,1] \), lo que nos permite interpretarla como una probabilidad.

---

### **2. ¿Qué se espera obtener con la Regresión Logística?**
Al aplicar este modelo, buscamos:

- **Identificar las variables más influyentes en la calidad del vino.**  
  - ¿El contenido de alcohol tiene una relación directa con la calidad?  
  - ¿Factores como la acidez o el pH impactan en la percepción del vino?  

- **Predecir si un vino es de alta o baja calidad basado en sus características fisicoquímicas.**  
  - La salida del modelo será una **probabilidad**.  
  - Si \( P(y=1) \) es mayor a un umbral (generalmente 0.5), clasificamos el vino como **alta calidad** (`1`).  

- **Evaluar la precisión del modelo** mediante métricas como:
  - **Exactitud (Accuracy):** Porcentaje de predicciones correctas.  
  - **Matriz de Confusión:** Para visualizar falsos positivos y falsos negativos.  
  - **ROC-AUC Score:** Para medir qué tan bien el modelo distingue entre las clases.  

---

### **3. Diferencias con la Regresión Lineal**
| Característica            | Regresión Lineal                        | Regresión Logística                      |
|---------------------------|----------------------------------------|-----------------------------------------|
| **Tipo de variable objetivo** | Continua (números reales)             | Categórica (binaria: 0 o 1)             |
| **Función aplicada**       | Línea recta                           | Función sigmoide                       |
| **Salida del modelo**      | Valor numérico                        | Probabilidad de pertenecer a una clase |
| **Interpretación**         | Cambios absolutos en `y` por `X_i`    | Probabilidad de pertenecer a `y=1`     |

---

### **4. Aplicación en la Calidad del Vino**
En este análisis, la **Regresión Logística** nos ayudará a:

- **Determinar qué características del vino están asociadas con una alta calidad (`quality > 7`).**  
- **Predecir si un vino es de alta calidad a partir de sus características fisicoquímicas.**  
- **Evaluar si los vinos tintos y blancos presentan diferencias significativas en su calidad percibida.**  
 


In [ ]:
# Primero preparamos los datos y determinamos qué caracteriza a un vino de buena o mala calidad
# Si quality > 7 le damos valor 1, si quality ≤ 7 le damos valor 0

# Copiamos el dataframe original
df_wine_predictive_quality = df_wine_quality_winsorized.copy()

# Aplicamos la transformación binaria a quality
df_wine_predictive_quality['quality_binary'] = df_wine_predictive_quality['quality'].apply(lambda x: 1 if x >= 7 else 0)

# Verificamos el resultado con un muestreo
print(df_wine_predictive_quality[['quality', 'quality_binary']].sample(5))

# Verificamos la distribución de las clases
print(df_wine_predictive_quality['quality_binary'].value_counts())




### **Observaciones sobre la Distribución de Clases**
Podemos observar un **desbalance en los datos**:
- La mayoría de los vinos tienen **baja calidad** (`quality_binary = 0`), con **5220 muestras**.
- Solo **1277 muestras** corresponden a vinos de **alta calidad** (`quality_binary = 1`).
- Esto implica que **aproximadamente el 30% de los datos pertenecen a la clase minoritaria** (`quality > 7`).

### **Impacto del Desbalance en el Modelo**
El desbalance de clases puede afectar el rendimiento del modelo de clasificación:
- **Predicciones sesgadas:** Si una clase es mucho más frecuente que la otra, el modelo puede aprender a predecir mayormente la clase más común (`0`), ignorando la minoritaria.
- **Riesgo de baja capacidad de generalización:** Un modelo altamente sesgado podría no identificar correctamente los vinos de alta calidad.
- **Métricas engañosas:** La exactitud (`accuracy`) del modelo podría parecer alta si simplemente predice la clase mayoritaria, sin realmente capturar patrones útiles en los datos.

---

### División en Conjuntos de Entrenamiento y Prueba
Ahora vamos a dividir nuestro conjunto de datos en entrenamiento (80%) y prueba (20%). Esto nos permitirá entrenar el modelo con la mayor parte de los datos y luego evaluar su desempeño en datos que no ha visto antes.

##### ¿Por qué hacemos esta división?
Evitar el sobreajuste `(overfitting)`: Si entrenamos y evaluamos el modelo en los mismos datos, este podría "memorizar" los datos en lugar de aprender patrones generales.
**Evaluación realista**: Al usar datos de prueba que no fueron usados para entrenar, obtenemos una mejor idea de cómo se comportará el modelo en situaciones reales.
Generalización: Nos aseguramos de que el modelo pueda hacer predicciones en datos nuevos y no solo en los datos de entrenamiento.


In [ ]:
# Convertimos 'type-wine' en variables numéricas con One-Hot Encoding
df_encoded = pd.get_dummies(df_wine_predictive_quality, columns=["type-wine"], drop_first=True)

# Separar X e y
X = df_encoded.drop(columns=["quality", "quality_binary"])
y = df_encoded["quality_binary"]

# Verificamos la estructura de X e y
print(X.shape, y.shape)


¿Qué hace pd.get_dummies()?
Convierte variables categóricas en variables numéricas.
Como type-wine tiene dos valores posibles ("red" y "white"), se crea una columna binaria.
drop_first=True elimina una de las categorías para evitar colinealidad en el modelo.

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificamos las dimensiones de los conjuntos resultantes
print(f"Tamaño de X_train: {X_train.shape}, Tamaño de y_train: {y_train.shape}")
print(f"Tamaño de X_test: {X_test.shape}, Tamaño de y_test: {y_test.shape}")


Se realizó una división de los datos en entrenamiento (80%) y prueba (20%).
X_train (entrenamiento): 5197 muestras con 14 características.
X_test (prueba): 1300 muestras con 14 características.
y_train: 5197 etiquetas correspondientes a los datos de entrenamiento.
y_test: 1300 etiquetas para evaluar el modelo

##### Primero entrenaremos a nuestro modelo

In [ ]:
print(X_train.dtypes)

In [ ]:
X_train = X_train.drop(columns=["array_column", "date_column"])
X_test = X_test.drop(columns=["array_column", "date_column"])


In [ ]:
X_train["type-wine_white"] = X_train["type-wine_white"].astype(int)
X_test["type-wine_white"] = X_test["type-wine_white"].astype(int)

In [ ]:
print(X_train.dtypes)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Crear el modelo de regresión logística
model = LogisticRegression(max_iter=1000, random_state=42)

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

##### Error: El optimizador L-BFGS no ha convergido

Al entrenar el modelo de **Regresión Logística**, se ha producido el siguiente error:

> **El optimizador L-BFGS no ha logrado converger en el número máximo de iteraciones (`max_iter=1000`)**, lo que significa que el modelo no ha podido encontrar una solución óptima.

##### **¿Por qué ocurre este problema?**
Existen varias razones por las cuales la optimización de la Regresión Logística puede no converger:

1. **Los datos no están escalados**  
   - La **Regresión Logística es sensible a las escalas de las variables**.  
   - Si las características tienen rangos muy diferentes (por ejemplo, `pH` entre **3-4** y `sulfatos` entre **0.3-1.5**), el optimizador puede tener problemas para converger.  
   - **Solución:** Aplicar escalado estándar con `StandardScaler` de `sklearn`.  

2. **Pocas iteraciones (`max_iter=1000`)**  
   - En algunos casos, el modelo necesita más iteraciones para encontrar la mejor solución.  
   - **Solución:** Aumentar el número de iteraciones (`max_iter=5000` o más) para permitir que el optimizador siga buscando una solución óptima.  

3. **Colinealidad o valores extremos en las variables**  
   - Algunas variables pueden estar altamente correlacionadas, lo que puede dificultar el entrenamiento del modelo.  
   - La presencia de **valores extremos (outliers)** también puede hacer que la optimización no sea estable.  
   - **Solución:**  
     - Revisar la matriz de correlación para detectar variables redundantes.  
     - Aplicar técnicas como **reducción de dimensionalidad** (PCA) o **eliminación de variables altamente correlacionadas**.  

Para evitar este problemavamos a normalizar los datos con StandardScaler.

La mejor práctica en regresión logística es escalar los datos para que todas las variables tengan una media de 0 y una desviación estándar de 1:


In [ ]:
from sklearn.preprocessing import StandardScaler

# Aplicar StandardScaler a los datos de entrada
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Crear el modelo de regresión logística
model = LogisticRegression(max_iter=2000, random_state=42, class_weight="balanced")

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1-Score: {f1:.4f}")


### Evaluación del Modelo de Regresión Logística

Hemos probado dos versiones del modelo de **Regresión Logística**:  
1. **Sin balanceo de clases (`class_weight` por defecto)**  
2.  **Con balanceo automático de clases (`class_weight='balanced'`)**

A continuación, se presentan los resultados:

| Modelo | Accuracy | F1-Score |
|--------|----------|----------|
| **Sin balanceo (`class_weight` por defecto)** | 0.8331 | 0.4119 |
| **Con balanceo (`class_weight='balanced'`)** | 0.7123 | 0.5053 |

##### **Análisis de los resultados**
- **Sin balanceo (`class_weight` por defecto)**
  - Se obtiene una **alta exactitud (accuracy = 83.31%)**, pero esto es engañoso.
  - El modelo prioriza la **clase mayoritaria** (vinos de baja calidad) y tiene dificultades para identificar los vinos de alta calidad.
  - Esto se refleja en un **F1-Score bajo (0.4119)**, indicando que la capacidad del modelo para detectar correctamente la clase minoritaria es deficiente.

- **Con balanceo (`class_weight='balanced'`)**
  - La **accuracy baja a 71.23%**, ya que el modelo ya no predice solo la clase mayoritaria.
  - Sin embargo, el **F1-Score mejora a 0.5053**, lo que significa que ahora el modelo es más efectivo en la detección de vinos de alta calidad.
  - Aunque la exactitud general ha disminuido, el modelo ahora tiene una mejor capacidad de generalización para ambas clases.

---

In [ ]:
from sklearn.metrics import confusion_matrix

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar la matriz de confusión
print("Matriz de Confusión:")
print(conf_matrix)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Visualizar la matriz de confusión
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Baja Calidad", "Alta Calidad"], yticklabels=["Baja Calidad", "Alta Calidad"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de Confusión")
plt.show()


#### **Visualización de la Matriz de Confusión**
Para analizar mejor el rendimiento del modelo, utilizaremos la **Matriz de Confusión**, que nos permite visualizar cómo se están clasificando las muestras:

| **Predicción Negativa (0)** | **Predicción Positiva (1)** |
|-----------------------------|-----------------------------|
| **Real Negativa (0)** | **TN (Verdaderos Negativos)** | **FP (Falsos Positivos)** |
| **Real Positiva (1)** | **FN (Falsos Negativos)** | **TP (Verdaderos Positivos)** |

#### **¿Qué significa cada término?**
- **TN (Verdaderos Negativos):** Vinos de baja calidad correctamente clasificados como `0`.
- **FP (Falsos Positivos):** Vinos de baja calidad incorrectamente clasificados como `1`.
- **FN (Falsos Negativos):** Vinos de alta calidad incorrectamente clasificados como `0`.
- **TP (Verdaderos Positivos):** Vinos de alta calidad correctamente clasificados como `1`.

---

#### **Conclusión**
- **Si nuestro objetivo es identificar con precisión los vinos de alta calidad**, el **F1-Score** es la métrica más importante, ya que equilibra precisión y recall.
- **El balanceo de clases (`class_weight='balanced'`) ayuda a mejorar la detección de la clase minoritaria**, aunque a costa de reducir la exactitud global.
- **El siguiente paso** es visualizar la matriz de confusión para entender mejor dónde el modelo está fallando y ajustar hiperparámetros o probar otros algoritmos (p. ej., **árboles de decisión o Random Forest**) para mejorar el rendimiento.


In [ ]:
#  Predecimos probabilidades en vez de clases
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # Tomamos solo la probabilidad de la clase positiva (1)

#  Ajustamos el umbral de clasificación a 0.6 (más conservador)
threshold = 0.6
y_pred_adjusted = (y_proba > threshold).astype(int)

#  Evaluamos las métricas de clasificación con el nuevo umbral
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

accuracy = accuracy_score(y_test, y_pred_adjusted)
f1 = f1_score(y_test, y_pred_adjusted)

print(f" Accuracy: {accuracy:.4f}")
print(f" F1-Score: {f1:.4f}")

#  Matriz de confusión con el nuevo umbral
conf_matrix = confusion_matrix(y_test, y_pred_adjusted)

#  Visualización de la matriz de confusión
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=["Baja Calidad", "Alta Calidad"], yticklabels=["Baja Calidad", "Alta Calidad"])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title(f"Matriz de Confusión (Umbral {threshold})")
plt.show()


Hemos evaluado el modelo con la **matriz de confusión estándar (umbral 0.5)** para obtener una referencia inicial, pero observamos que tenía dificultades para identificar los vinos de alta calidad. Por ello, ajustamos el umbral a **0.6** para mejorar la precisión en la detección de la clase minoritaria, aunque los resultados siguen sin ser óptimos. Dado que la Regresión Logística no logra una separación clara entre clases, ahora probaremos un **DecisionTreeClassifier**, que puede capturar relaciones no lineales en los datos y mejorar la clasificación. 


### Predicción de la Calidad del Vino utilizando Árboles de Decisión

Hasta ahora, hemos intentado clasificar los vinos en categorías de **alta y baja calidad** utilizando **Regresión Logística**, pero los resultados han demostrado que este enfoque no logra capturar completamente la complejidad de los datos. Además, ajustar el umbral de clasificación solo ha generado mejoras marginales en la detección de la clase minoritaria.

Por ello, en este paso, exploraremos un enfoque diferente: **predecir directamente la calidad del vino (quality) como una variable categórica**, en lugar de una simple clasificación binaria. Para ello, utilizaremos **Árboles de Decisión**, una técnica de Machine Learning que permite capturar relaciones no lineales entre las características físico-químicas del vino y su calidad.

Los Árboles de Decisión tienen varias ventajas en este contexto:
- Son interpretables y permiten entender qué factores influyen más en la calidad del vino.  
- Pueden manejar relaciones no lineales entre las variables.  
- No requieren que las características sean escaladas, a diferencia de la Regresión Logística.  

En este proceso, tomaremos los valores originales de la variable `quality` y entrenaremos un **DecisionTreeClassifier** para analizar si podemos encontrar patrones más representativos y mejorar la predicción de la calidad del vino. 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 1️ Copiar el dataset original para no modificarlo
df_wine_copy = df_wine_quality.copy()

# 2️ Seleccionar solo las características más relevantes según la importancia de variables previa
selected_features = ["chlorides", "residual sugar", "free sulfur dioxide", 
                     "density", "pH", "total sulfur dioxide"]
X_selected = df_wine_copy[selected_features]
y = df_wine_copy["quality"]

# 3️ Codificar la variable objetivo `y` (ya que tiene valores categóricos)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 4️ Dividir en conjunto de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_encoded, test_size=0.2, random_state=42)

# 5️ Normalizar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 6️ Entrenar el modelo con el mejor ajuste
best_tree_model = DecisionTreeClassifier(max_depth=10, random_state=42)
best_tree_model.fit(X_train_scaled, y_train)

# 7️ Obtener predicciones en el conjunto de prueba
y_pred = best_tree_model.predict(X_test_scaled)

# 8️ Evaluar el modelo con métricas de error
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f" Error Absoluto Medio (MAE): {mae:.4f}")
print(f" Error Cuadrático Medio (MSE): {mse:.4f}")
print(f" Raíz del Error Cuadrático Medio (RMSE): {rmse:.4f}")

# 9️ Evaluación con margen de error ±1 (es decir, permitir que el modelo prediga con un pequeño error)
total = len(y_test)
aciertos_relajados = ((y_pred >= (y_test - 1)) & (y_pred <= (y_test + 1))).sum()
errores_relajados = total - aciertos_relajados

# Mostrar resultados
print(f"Número total de muestras: {total}")
print(f"Aciertos dentro del margen ±1: {aciertos_relajados} ({(aciertos_relajados / total) * 100:.2f}%)")
print(f"Errores fuera del margen ±1: {errores_relajados} ({(errores_relajados / total) * 100:.2f}%)")

#  Visualizar Importancia de Características
importances = best_tree_model.feature_importances_
importance_df = pd.DataFrame({"Característica": X_selected.columns, "Importancia": importances})
importance_df = importance_df.sort_values(by="Importancia", ascending=False)

plt.figure(figsize=(10,5))
sns.barplot(x="Importancia", y="Característica", data=importance_df)
plt.title(" Importancia de Características en el Árbol de Decisión")
plt.show()


# Conclusión Final del Proyecto

En este análisis sobre la **calidad del vino**, hemos aplicado distintos modelos de **aprendizaje automático** para predecir la variable `quality`. A lo largo del proceso, hemos llevado a cabo experimentos, ajustes y análisis detallados que nos han permitido extraer conclusiones clave sobre los factores que influyen en la calidad del vino y la capacidad de los modelos para predecirla.

---

##  Preprocesamiento y Selección de Variables

- Inicialmente, seleccionamos las características más relevantes basándonos en su importancia en distintos modelos:
  - `chlorides`, `residual sugar`, `free sulfur dioxide`, `density`, `pH`, `total sulfur dioxide`.
- Observamos que la relevancia de estas variables varía según el modelo utilizado, lo que sugiere que **la calidad del vino no depende de un único patrón objetivo**, sino de una combinación de factores.

---

##  Rendimiento de los Modelos

Probamos varios modelos de Machine Learning para evaluar su capacidad predictiva:

| Modelo            | Precisión Promedio |
|------------------|------------------|
| **Árbol de Decisión** | 38.62% |
| **Random Forest** | 42.76% |
| **XGBoost**      | 43.05% |

- **Árbol de Decisión**: Ajustamos la profundidad del árbol (`max_depth`) y observamos que valores bajos llevaban a un **modelo poco preciso**, mientras que valores altos generaban **sobreajuste**.
- **Random Forest**: Redujo los errores y mejoró la capacidad predictiva en comparación con el Árbol de Decisión.
- **XGBoost**: No pudo evaluarse completamente debido a incompatibilidades con la variable `y`, aunque en otros estudios ha mostrado un rendimiento superior.

---

##  Evaluación de Resultados

- **La precisión del mejor modelo no superó el 58%**, lo que indica que la variable `quality` tiene un alto grado de subjetividad.
- **Errores de Predicción**:
  -  **Error Absoluto Medio (MAE)**: `0.5754`
  -  **Error Cuadrático Medio (MSE)**: `0.7646`
  -  **Raíz del Error Cuadrático Medio (RMSE)**: `0.8744`
- **Implementamos un margen de error de ±1 unidad** en la predicción y descubrimos que el modelo **acierta en un 92.08% de los casos**, lo que indica que es más robusto de lo que parecía en un inicio.

---

##  Reflexión sobre la Predicción de Calidad del Vino

1. **La calidad del vino no es un concepto estrictamente objetivo**. Diferentes modelos destacan distintas variables como más importantes, lo que sugiere que **no hay una única fórmula matemática** para definir la calidad.
2. **El hecho de que modelos distintos otorguen pesos diferentes a las variables** refuerza la idea de que la calidad del vino **depende en gran medida de la percepción de los catadores** y no solo de características físico-químicas.
3. **Los modelos de Machine Learning pueden ser útiles para identificar tendencias generales**, pero **no reemplazan el criterio humano** en la evaluación de calidad del vino.

---

##  Conclusión Final

- **El modelo de Árbol de Decisión con margen de error de ±1 unidad obtuvo un 92.08% de precisión**, lo que demuestra que puede ser útil en la predicción aproximada de la calidad del vino.
- **El problema es altamente subjetivo**, lo que dificulta alcanzar una alta precisión con modelos de Machine Learning.
- **La combinación de análisis estadístico y técnicas de IA puede ayudar a identificar patrones útiles**, aunque la evaluación final de la calidad del vino seguirá dependiendo de expertos humanos.

 **Próximos pasos**: Se podrían explorar **modelos más avanzados**, como redes neuronales o modelos híbridos, que combinen información química con datos sensoriales de catadores.

---

 **Resumen en una frase**:  
**No hay una única ecuación para la calidad del vino, pero el análisis de datos nos ha permitido entender mejor los factores que influyen en su percepción.** 
